# Wood Beam Clips

In [1]:
from functools import partial
from itertools import product
from IPython.display import Markdown
import numpy as np
import pandas as pd
from wsweng.wood.dowels import wood_bolt

## Givens

In [104]:
diameters = [0.625, 0.75, 0.875, 1.0]  # Nominal diameter, inches
thicknesses = [1.50, 1.75, 3.00, 3.50, 5.125, 5.25, 5.50, 7.0]
bolt_counts = range(2, 10, 1)
spacings = [2.5, 3, 4]
clip_thickness = 1/4.0
species = "DFL"
steel = "A36"
make_bolt = partial(
    wood_bolt,
    full_diameter=False,
    main_material=species,
    side_material=steel,
    side_thickness=clip_thickness
)

### Generate design permutations

Find all permutations of diameter, thickness, N and double shear:

In [117]:
p_dia = product(diameters, thicknesses, bolt_counts, spacings, [True, False])
bolts = pd.DataFrame(data=p_dia, columns=["d", "t", "n", "s", "ds"])
print(bolts.round(2).sample(3))

         d     t  n    s     ds
1114  0.88  7.00  3  4.0   True
978   0.88  5.12  5  2.5   True
1147  0.88  7.00  9  2.5  False


### Calculate capacity

In [118]:
_z0 = [make_bolt(_d, _t, double_shear=_ds).Zv(theta=90.0)
               for _d, _t, _ds in zip(bolts["d"], bolts["t"], bolts["ds"])]
_c_spcg = np.clip(bolts["s"]/(4*bolts["d"]), 0, 1.0)
bolts["c_spcg"] = np.where(_c_spcg < 0.5, 0, _c_spcg)  # Zero capacity for less than min spacing.
bolts["z"] = bolts["n"]*_c_spcg*_z0
print(bolts.round(2).sample(3))

         d     t  n    s     ds  c_spcg        z
665   0.75  5.25  8  4.0  False    1.00  5482.08
1012  0.88  5.25  2  4.0   True    1.00  3534.44
1267  1.00  3.00  5  2.5  False    0.62  1883.97


### Geometry

In [119]:
bolts["h_beam"] = (bolts["n"] - 1)*bolts["s"] + 2*4*bolts["d"]
bolts["h_clip"] = (bolts["n"] - 1)*bolts["s"] + 3.0
bolts["d_end"] = np.clip(1.5*bolts["d"], 1.25, np.inf)

print(bolts.round(2).sample(3))

         d    t  n    s     ds  c_spcg        z  h_beam  h_clip  d_end
695   0.75  5.5  5  4.0  False    1.00  3426.30    22.0    19.0   1.25
1123  0.88  7.0  5  2.5  False    0.71  3155.75    17.0    13.0   1.31
1265  1.00  3.0  4  4.0  False    1.00  2411.48    20.0    15.0   1.50


## Schedules

In [120]:
full_sched = bolts.pivot(
    index=["ds", "s", "d", "n", "h_beam", "h_clip"],
    columns=["t"],
    values="z"
).reset_index(("n", "h_beam", "h_clip"))
print(full_sched.head(3).round(1))

t                n  h_beam  h_clip     1.5    1.75     3.0     3.5   5.125  \
ds    s   d                                                                  
False 2.5 0.625  2     7.5     5.5   603.3   640.4   889.6  1004.2  1021.8   
          0.625  3    10.0     8.0   904.9   960.6  1334.4  1506.3  1532.7   
          0.625  4    12.5    10.5  1206.5  1280.8  1779.2  2008.5  2043.5   

t                  5.25     5.5     7.0  
ds    s   d                              
False 2.5 0.625  1021.8  1021.8  1021.8  
          0.625  1532.7  1532.7  1532.7  
          0.625  2043.5  2043.5  2043.5  


In [121]:
from tabulate import tabulate
def schedule(d) -> str:
    return tabulate(
        d,
        headers="keys",
        tablefmt="pipe", # "pipe", "rst", "latex"
        numalign="decimal",
        colalign=("center", "center", "center",),
        floatfmt=".1f",
        showindex=False,
    )

In [122]:
_out = []
for _d, dd in full_sched.groupby("d"):
    _out.append(f"### {_d:.3f}\" Diameter\n\n")
    for _s, ds in dd.groupby("s"):
        _out.append(f"#### {_s:.2f}\" oc. Spacing\n\n")
        for _ds, dv in ds.groupby("ds"):
            _out.append(f"##### {'Double Shear' if _ds else 'Single Shear'}\n\n")
            _out.append(schedule(dv))
            _out.append("\n\n")

Markdown("".join(_out))

### 0.625" Diameter

#### 2.50" oc. Spacing

##### Single Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   7.5    |   5.5    |  603.3 |  640.4 |  889.6 | 1004.2 |  1021.8 | 1021.8 | 1021.8 | 1021.8 |
| 3.0 |   10.0   |   8.0    |  904.9 |  960.6 | 1334.4 | 1506.3 |  1532.7 | 1532.7 | 1532.7 | 1532.7 |
| 4.0 |   12.5   |   10.5   | 1206.5 | 1280.8 | 1779.2 | 2008.5 |  2043.5 | 2043.5 | 2043.5 | 2043.5 |
| 5.0 |   15.0   |   13.0   | 1508.2 | 1601.0 | 2224.0 | 2510.6 |  2554.4 | 2554.4 | 2554.4 | 2554.4 |
| 6.0 |   17.5   |   15.5   | 1809.8 | 1921.3 | 2668.8 | 3012.7 |  3065.3 | 3065.3 | 3065.3 | 3065.3 |
| 7.0 |   20.0   |   18.0   | 2111.4 | 2241.5 | 3113.6 | 3514.8 |  3576.2 | 3576.2 | 3576.2 | 3576.2 |
| 8.0 |   22.5   |   20.5   | 2413.1 | 2561.7 | 3558.4 | 4016.9 |  4087.1 | 4087.1 | 4087.1 | 4087.1 |
| 9.0 |   25.0   |   23.0   | 2714.7 | 2881.9 | 4003.1 | 4519.0 |  4598.0 | 4598.0 | 4598.0 | 4598.0 |

##### Double Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   7.5    |   5.5    |  871.0 | 1016.1 | 1742.0 | 2032.3 |  2043.5 | 2043.5 | 2043.5 | 2043.5 |
| 3.0 |   10.0   |   8.0    | 1306.5 | 1524.2 | 2613.0 | 3048.4 |  3065.3 | 3065.3 | 3065.3 | 3065.3 |
| 4.0 |   12.5   |   10.5   | 1742.0 | 2032.3 | 3483.9 | 4064.6 |  4087.1 | 4087.1 | 4087.1 | 4087.1 |
| 5.0 |   15.0   |   13.0   | 2177.5 | 2540.4 | 4354.9 | 5080.7 |  5108.9 | 5108.9 | 5108.9 | 5108.9 |
| 6.0 |   17.5   |   15.5   | 2613.0 | 3048.4 | 5225.9 | 6096.9 |  6130.6 | 6130.6 | 6130.6 | 6130.6 |
| 7.0 |   20.0   |   18.0   | 3048.4 | 3556.5 | 6096.9 | 7113.0 |  7152.4 | 7152.4 | 7152.4 | 7152.4 |
| 8.0 |   22.5   |   20.5   | 3483.9 | 4064.6 | 6967.9 | 8129.2 |  8174.2 | 8174.2 | 8174.2 | 8174.2 |
| 9.0 |   25.0   |   23.0   | 3919.4 | 4572.7 | 7838.9 | 9145.3 |  9196.0 | 9196.0 | 9196.0 | 9196.0 |

#### 3.00" oc. Spacing

##### Single Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   8.0    |   6.0    |  603.3 |  640.4 |  889.6 | 1004.2 |  1021.8 | 1021.8 | 1021.8 | 1021.8 |
| 3.0 |   11.0   |   9.0    |  904.9 |  960.6 | 1334.4 | 1506.3 |  1532.7 | 1532.7 | 1532.7 | 1532.7 |
| 4.0 |   14.0   |   12.0   | 1206.5 | 1280.8 | 1779.2 | 2008.5 |  2043.5 | 2043.5 | 2043.5 | 2043.5 |
| 5.0 |   17.0   |   15.0   | 1508.2 | 1601.0 | 2224.0 | 2510.6 |  2554.4 | 2554.4 | 2554.4 | 2554.4 |
| 6.0 |   20.0   |   18.0   | 1809.8 | 1921.3 | 2668.8 | 3012.7 |  3065.3 | 3065.3 | 3065.3 | 3065.3 |
| 7.0 |   23.0   |   21.0   | 2111.4 | 2241.5 | 3113.6 | 3514.8 |  3576.2 | 3576.2 | 3576.2 | 3576.2 |
| 8.0 |   26.0   |   24.0   | 2413.1 | 2561.7 | 3558.4 | 4016.9 |  4087.1 | 4087.1 | 4087.1 | 4087.1 |
| 9.0 |   29.0   |   27.0   | 2714.7 | 2881.9 | 4003.1 | 4519.0 |  4598.0 | 4598.0 | 4598.0 | 4598.0 |

##### Double Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   8.0    |   6.0    |  871.0 | 1016.1 | 1742.0 | 2032.3 |  2043.5 | 2043.5 | 2043.5 | 2043.5 |
| 3.0 |   11.0   |   9.0    | 1306.5 | 1524.2 | 2613.0 | 3048.4 |  3065.3 | 3065.3 | 3065.3 | 3065.3 |
| 4.0 |   14.0   |   12.0   | 1742.0 | 2032.3 | 3483.9 | 4064.6 |  4087.1 | 4087.1 | 4087.1 | 4087.1 |
| 5.0 |   17.0   |   15.0   | 2177.5 | 2540.4 | 4354.9 | 5080.7 |  5108.9 | 5108.9 | 5108.9 | 5108.9 |
| 6.0 |   20.0   |   18.0   | 2613.0 | 3048.4 | 5225.9 | 6096.9 |  6130.6 | 6130.6 | 6130.6 | 6130.6 |
| 7.0 |   23.0   |   21.0   | 3048.4 | 3556.5 | 6096.9 | 7113.0 |  7152.4 | 7152.4 | 7152.4 | 7152.4 |
| 8.0 |   26.0   |   24.0   | 3483.9 | 4064.6 | 6967.9 | 8129.2 |  8174.2 | 8174.2 | 8174.2 | 8174.2 |
| 9.0 |   29.0   |   27.0   | 3919.4 | 4572.7 | 7838.9 | 9145.3 |  9196.0 | 9196.0 | 9196.0 | 9196.0 |

#### 4.00" oc. Spacing

##### Single Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   9.0    |   7.0    |  603.3 |  640.4 |  889.6 | 1004.2 |  1021.8 | 1021.8 | 1021.8 | 1021.8 |
| 3.0 |   13.0   |   11.0   |  904.9 |  960.6 | 1334.4 | 1506.3 |  1532.7 | 1532.7 | 1532.7 | 1532.7 |
| 4.0 |   17.0   |   15.0   | 1206.5 | 1280.8 | 1779.2 | 2008.5 |  2043.5 | 2043.5 | 2043.5 | 2043.5 |
| 5.0 |   21.0   |   19.0   | 1508.2 | 1601.0 | 2224.0 | 2510.6 |  2554.4 | 2554.4 | 2554.4 | 2554.4 |
| 6.0 |   25.0   |   23.0   | 1809.8 | 1921.3 | 2668.8 | 3012.7 |  3065.3 | 3065.3 | 3065.3 | 3065.3 |
| 7.0 |   29.0   |   27.0   | 2111.4 | 2241.5 | 3113.6 | 3514.8 |  3576.2 | 3576.2 | 3576.2 | 3576.2 |
| 8.0 |   33.0   |   31.0   | 2413.1 | 2561.7 | 3558.4 | 4016.9 |  4087.1 | 4087.1 | 4087.1 | 4087.1 |
| 9.0 |   37.0   |   35.0   | 2714.7 | 2881.9 | 4003.1 | 4519.0 |  4598.0 | 4598.0 | 4598.0 | 4598.0 |

##### Double Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   9.0    |   7.0    |  871.0 | 1016.1 | 1742.0 | 2032.3 |  2043.5 | 2043.5 | 2043.5 | 2043.5 |
| 3.0 |   13.0   |   11.0   | 1306.5 | 1524.2 | 2613.0 | 3048.4 |  3065.3 | 3065.3 | 3065.3 | 3065.3 |
| 4.0 |   17.0   |   15.0   | 1742.0 | 2032.3 | 3483.9 | 4064.6 |  4087.1 | 4087.1 | 4087.1 | 4087.1 |
| 5.0 |   21.0   |   19.0   | 2177.5 | 2540.4 | 4354.9 | 5080.7 |  5108.9 | 5108.9 | 5108.9 | 5108.9 |
| 6.0 |   25.0   |   23.0   | 2613.0 | 3048.4 | 5225.9 | 6096.9 |  6130.6 | 6130.6 | 6130.6 | 6130.6 |
| 7.0 |   29.0   |   27.0   | 3048.4 | 3556.5 | 6096.9 | 7113.0 |  7152.4 | 7152.4 | 7152.4 | 7152.4 |
| 8.0 |   33.0   |   31.0   | 3483.9 | 4064.6 | 6967.9 | 8129.2 |  8174.2 | 8174.2 | 8174.2 | 8174.2 |
| 9.0 |   37.0   |   35.0   | 3919.4 | 4572.7 | 7838.9 | 9145.3 |  9196.0 | 9196.0 | 9196.0 | 9196.0 |

### 0.750" Diameter

#### 2.50" oc. Spacing

##### Single Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   8.5    |   5.5    |  580.2 |  612.7 |  837.6 |  942.5 |  1142.1 | 1142.1 | 1142.1 | 1142.1 |
| 3.0 |   11.0   |   8.0    |  870.3 |  919.0 | 1256.4 | 1413.8 |  1713.1 | 1713.1 | 1713.1 | 1713.1 |
| 4.0 |   13.5   |   10.5   | 1160.5 | 1225.3 | 1675.2 | 1885.1 |  2284.2 | 2284.2 | 2284.2 | 2284.2 |
| 5.0 |   16.0   |   13.0   | 1450.6 | 1531.7 | 2094.0 | 2356.3 |  2855.2 | 2855.2 | 2855.2 | 2855.2 |
| 6.0 |   18.5   |   15.5   | 1740.7 | 1838.0 | 2512.8 | 2827.6 |  3426.3 | 3426.3 | 3426.3 | 3426.3 |
| 7.0 |   21.0   |   18.0   | 2030.8 | 2144.3 | 2931.6 | 3298.9 |  3997.3 | 3997.3 | 3997.3 | 3997.3 |
| 8.0 |   23.5   |   20.5   | 2320.9 | 2450.7 | 3350.4 | 3770.1 |  4568.4 | 4568.4 | 4568.4 | 4568.4 |
| 9.0 |   26.0   |   23.0   | 2611.0 | 2757.0 | 3769.2 | 4241.4 |  5139.4 | 5139.4 | 5139.4 | 5139.4 |

##### Double Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |    5.25 |     5.5 |     7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|--------:|--------:|--------:|
| 2.0 |   8.5    |   5.5    |  808.2 |  943.0 | 1616.5 | 1885.9 |  2284.2 |  2284.2 |  2284.2 |  2284.2 |
| 3.0 |   11.0   |   8.0    | 1212.4 | 1414.4 | 2424.7 | 2828.9 |  3426.3 |  3426.3 |  3426.3 |  3426.3 |
| 4.0 |   13.5   |   10.5   | 1616.5 | 1885.9 | 3233.0 | 3771.8 |  4568.4 |  4568.4 |  4568.4 |  4568.4 |
| 5.0 |   16.0   |   13.0   | 2020.6 | 2357.4 | 4041.2 | 4714.8 |  5710.5 |  5710.5 |  5710.5 |  5710.5 |
| 6.0 |   18.5   |   15.5   | 2424.7 | 2828.9 | 4849.5 | 5657.7 |  6852.6 |  6852.6 |  6852.6 |  6852.6 |
| 7.0 |   21.0   |   18.0   | 2828.9 | 3300.3 | 5657.7 | 6600.7 |  7994.7 |  7994.7 |  7994.7 |  7994.7 |
| 8.0 |   23.5   |   20.5   | 3233.0 | 3771.8 | 6466.0 | 7543.6 |  9136.8 |  9136.8 |  9136.8 |  9136.8 |
| 9.0 |   26.0   |   23.0   | 3637.1 | 4243.3 | 7274.2 | 8486.6 | 10278.9 | 10278.9 | 10278.9 | 10278.9 |

#### 3.00" oc. Spacing

##### Single Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   9.0    |   6.0    |  696.3 |  735.2 | 1005.1 | 1131.0 |  1370.5 | 1370.5 | 1370.5 | 1370.5 |
| 3.0 |   12.0   |   9.0    | 1044.4 | 1102.8 | 1507.7 | 1696.6 |  2055.8 | 2055.8 | 2055.8 | 2055.8 |
| 4.0 |   15.0   |   12.0   | 1392.5 | 1470.4 | 2010.3 | 2262.1 |  2741.0 | 2741.0 | 2741.0 | 2741.0 |
| 5.0 |   18.0   |   15.0   | 1740.7 | 1838.0 | 2512.8 | 2827.6 |  3426.3 | 3426.3 | 3426.3 | 3426.3 |
| 6.0 |   21.0   |   18.0   | 2088.8 | 2205.6 | 3015.4 | 3393.1 |  4111.6 | 4111.6 | 4111.6 | 4111.6 |
| 7.0 |   24.0   |   21.0   | 2437.0 | 2573.2 | 3517.9 | 3958.6 |  4796.8 | 4796.8 | 4796.8 | 4796.8 |
| 8.0 |   27.0   |   24.0   | 2785.1 | 2940.8 | 4020.5 | 4524.2 |  5482.1 | 5482.1 | 5482.1 | 5482.1 |
| 9.0 |   30.0   |   27.0   | 3133.2 | 3308.4 | 4523.1 | 5089.7 |  6167.3 | 6167.3 | 6167.3 | 6167.3 |

##### Double Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |     3.5 |   5.125 |    5.25 |     5.5 |     7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|--------:|--------:|--------:|--------:|--------:|
| 2.0 |   9.0    |   6.0    |  969.9 | 1131.5 | 1939.8 |  2263.1 |  2741.0 |  2741.0 |  2741.0 |  2741.0 |
| 3.0 |   12.0   |   9.0    | 1454.8 | 1697.3 | 2909.7 |  3394.6 |  4111.6 |  4111.6 |  4111.6 |  4111.6 |
| 4.0 |   15.0   |   12.0   | 1939.8 | 2263.1 | 3879.6 |  4526.2 |  5482.1 |  5482.1 |  5482.1 |  5482.1 |
| 5.0 |   18.0   |   15.0   | 2424.7 | 2828.9 | 4849.5 |  5657.7 |  6852.6 |  6852.6 |  6852.6 |  6852.6 |
| 6.0 |   21.0   |   18.0   | 2909.7 | 3394.6 | 5819.4 |  6789.3 |  8223.1 |  8223.1 |  8223.1 |  8223.1 |
| 7.0 |   24.0   |   21.0   | 3394.6 | 3960.4 | 6789.3 |  7920.8 |  9593.6 |  9593.6 |  9593.6 |  9593.6 |
| 8.0 |   27.0   |   24.0   | 3879.6 | 4526.2 | 7759.2 |  9052.3 | 10964.2 | 10964.2 | 10964.2 | 10964.2 |
| 9.0 |   30.0   |   27.0   | 4364.5 | 5091.9 | 8729.1 | 10183.9 | 12334.7 | 12334.7 | 12334.7 | 12334.7 |

#### 4.00" oc. Spacing

##### Single Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   10.0   |   7.0    |  696.3 |  735.2 | 1005.1 | 1131.0 |  1370.5 | 1370.5 | 1370.5 | 1370.5 |
| 3.0 |   14.0   |   11.0   | 1044.4 | 1102.8 | 1507.7 | 1696.6 |  2055.8 | 2055.8 | 2055.8 | 2055.8 |
| 4.0 |   18.0   |   15.0   | 1392.5 | 1470.4 | 2010.3 | 2262.1 |  2741.0 | 2741.0 | 2741.0 | 2741.0 |
| 5.0 |   22.0   |   19.0   | 1740.7 | 1838.0 | 2512.8 | 2827.6 |  3426.3 | 3426.3 | 3426.3 | 3426.3 |
| 6.0 |   26.0   |   23.0   | 2088.8 | 2205.6 | 3015.4 | 3393.1 |  4111.6 | 4111.6 | 4111.6 | 4111.6 |
| 7.0 |   30.0   |   27.0   | 2437.0 | 2573.2 | 3517.9 | 3958.6 |  4796.8 | 4796.8 | 4796.8 | 4796.8 |
| 8.0 |   34.0   |   31.0   | 2785.1 | 2940.8 | 4020.5 | 4524.2 |  5482.1 | 5482.1 | 5482.1 | 5482.1 |
| 9.0 |   38.0   |   35.0   | 3133.2 | 3308.4 | 4523.1 | 5089.7 |  6167.3 | 6167.3 | 6167.3 | 6167.3 |

##### Double Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |     3.5 |   5.125 |    5.25 |     5.5 |     7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|--------:|--------:|--------:|--------:|--------:|
| 2.0 |   10.0   |   7.0    |  969.9 | 1131.5 | 1939.8 |  2263.1 |  2741.0 |  2741.0 |  2741.0 |  2741.0 |
| 3.0 |   14.0   |   11.0   | 1454.8 | 1697.3 | 2909.7 |  3394.6 |  4111.6 |  4111.6 |  4111.6 |  4111.6 |
| 4.0 |   18.0   |   15.0   | 1939.8 | 2263.1 | 3879.6 |  4526.2 |  5482.1 |  5482.1 |  5482.1 |  5482.1 |
| 5.0 |   22.0   |   19.0   | 2424.7 | 2828.9 | 4849.5 |  5657.7 |  6852.6 |  6852.6 |  6852.6 |  6852.6 |
| 6.0 |   26.0   |   23.0   | 2909.7 | 3394.6 | 5819.4 |  6789.3 |  8223.1 |  8223.1 |  8223.1 |  8223.1 |
| 7.0 |   30.0   |   27.0   | 3394.6 | 3960.4 | 6789.3 |  7920.8 |  9593.6 |  9593.6 |  9593.6 |  9593.6 |
| 8.0 |   34.0   |   31.0   | 3879.6 | 4526.2 | 7759.2 |  9052.3 | 10964.2 | 10964.2 | 10964.2 | 10964.2 |
| 9.0 |   38.0   |   35.0   | 4364.5 | 5091.9 | 8729.1 | 10183.9 | 12334.7 | 12334.7 | 12334.7 | 12334.7 |

### 0.875" Diameter

#### 2.50" oc. Spacing

##### Single Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   9.5    |   5.5    |  560.0 |  588.7 |  793.8 |  890.6 |  1229.0 | 1256.0 | 1262.3 | 1262.3 |
| 3.0 |   12.0   |   8.0    |  840.1 |  883.0 | 1190.6 | 1336.0 |  1843.6 | 1883.9 | 1893.5 | 1893.5 |
| 4.0 |   14.5   |   10.5   | 1120.1 | 1177.4 | 1587.5 | 1781.3 |  2458.1 | 2511.9 | 2524.6 | 2524.6 |
| 5.0 |   17.0   |   13.0   | 1400.1 | 1471.7 | 1984.4 | 2226.6 |  3072.6 | 3139.9 | 3155.8 | 3155.8 |
| 6.0 |   19.5   |   15.5   | 1680.1 | 1766.0 | 2381.3 | 2671.9 |  3687.1 | 3767.9 | 3786.9 | 3786.9 |
| 7.0 |   22.0   |   18.0   | 1960.1 | 2060.4 | 2778.1 | 3117.3 |  4301.6 | 4395.9 | 4418.1 | 4418.1 |
| 8.0 |   24.5   |   20.5   | 2240.1 | 2354.7 | 3175.0 | 3562.6 |  4916.1 | 5023.9 | 5049.2 | 5049.2 |
| 9.0 |   27.0   |   23.0   | 2520.2 | 2649.1 | 3571.9 | 4007.9 |  5530.7 | 5651.8 | 5680.4 | 5680.4 |

##### Double Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |    5.25 |     5.5 |     7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|--------:|--------:|--------:|
| 2.0 |   9.5    |   5.5    |  756.0 |  882.0 | 1511.9 | 1763.9 |  2524.6 |  2524.6 |  2524.6 |  2524.6 |
| 3.0 |   12.0   |   8.0    | 1133.9 | 1322.9 | 2267.9 | 2645.9 |  3786.9 |  3786.9 |  3786.9 |  3786.9 |
| 4.0 |   14.5   |   10.5   | 1511.9 | 1763.9 | 3023.8 | 3527.8 |  5049.2 |  5049.2 |  5049.2 |  5049.2 |
| 5.0 |   17.0   |   13.0   | 1889.9 | 2204.9 | 3779.8 | 4409.8 |  6311.5 |  6311.5 |  6311.5 |  6311.5 |
| 6.0 |   19.5   |   15.5   | 2267.9 | 2645.9 | 4535.8 | 5291.7 |  7573.8 |  7573.8 |  7573.8 |  7573.8 |
| 7.0 |   22.0   |   18.0   | 2645.9 | 3086.8 | 5291.7 | 6173.7 |  8836.1 |  8836.1 |  8836.1 |  8836.1 |
| 8.0 |   24.5   |   20.5   | 3023.8 | 3527.8 | 6047.7 | 7055.6 | 10098.4 | 10098.4 | 10098.4 | 10098.4 |
| 9.0 |   27.0   |   23.0   | 3401.8 | 3968.8 | 6803.7 | 7937.6 | 11360.7 | 11360.7 | 11360.7 | 11360.7 |

#### 3.00" oc. Spacing

##### Single Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   10.0   |   6.0    |  672.0 |  706.4 |  952.5 | 1068.8 |  1474.8 | 1507.2 | 1514.8 | 1514.8 |
| 3.0 |   13.0   |   9.0    | 1008.1 | 1059.6 | 1428.8 | 1603.2 |  2212.3 | 2260.7 | 2272.1 | 2272.1 |
| 4.0 |   16.0   |   12.0   | 1344.1 | 1412.8 | 1905.0 | 2137.6 |  2949.7 | 3014.3 | 3029.5 | 3029.5 |
| 5.0 |   19.0   |   15.0   | 1680.1 | 1766.0 | 2381.3 | 2671.9 |  3687.1 | 3767.9 | 3786.9 | 3786.9 |
| 6.0 |   22.0   |   18.0   | 2016.1 | 2119.3 | 2857.5 | 3206.3 |  4424.5 | 4521.5 | 4544.3 | 4544.3 |
| 7.0 |   25.0   |   21.0   | 2352.2 | 2472.5 | 3333.8 | 3740.7 |  5162.0 | 5275.0 | 5301.7 | 5301.7 |
| 8.0 |   28.0   |   24.0   | 2688.2 | 2825.7 | 3810.0 | 4275.1 |  5899.4 | 6028.6 | 6059.0 | 6059.0 |
| 9.0 |   31.0   |   27.0   | 3024.2 | 3178.9 | 4286.3 | 4809.5 |  6636.8 | 6782.2 | 6816.4 | 6816.4 |

##### Double Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |    5.25 |     5.5 |     7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|--------:|--------:|--------:|
| 2.0 |   10.0   |   6.0    |  907.2 | 1058.3 | 1814.3 | 2116.7 |  3029.5 |  3029.5 |  3029.5 |  3029.5 |
| 3.0 |   13.0   |   9.0    | 1360.7 | 1587.5 | 2721.5 | 3175.0 |  4544.3 |  4544.3 |  4544.3 |  4544.3 |
| 4.0 |   16.0   |   12.0   | 1814.3 | 2116.7 | 3628.6 | 4233.4 |  6059.0 |  6059.0 |  6059.0 |  6059.0 |
| 5.0 |   19.0   |   15.0   | 2267.9 | 2645.9 | 4535.8 | 5291.7 |  7573.8 |  7573.8 |  7573.8 |  7573.8 |
| 6.0 |   22.0   |   18.0   | 2721.5 | 3175.0 | 5442.9 | 6350.1 |  9088.6 |  9088.6 |  9088.6 |  9088.6 |
| 7.0 |   25.0   |   21.0   | 3175.0 | 3704.2 | 6350.1 | 7408.4 | 10603.3 | 10603.3 | 10603.3 | 10603.3 |
| 8.0 |   28.0   |   24.0   | 3628.6 | 4233.4 | 7257.2 | 8466.8 | 12118.1 | 12118.1 | 12118.1 | 12118.1 |
| 9.0 |   31.0   |   27.0   | 4082.2 | 4762.6 | 8164.4 | 9525.1 | 13632.8 | 13632.8 | 13632.8 | 13632.8 |

#### 4.00" oc. Spacing

##### Single Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   11.0   |   7.0    |  784.1 |  824.2 | 1111.3 | 1246.9 |  1720.7 | 1758.3 | 1767.2 | 1767.2 |
| 3.0 |   15.0   |   11.0   | 1176.1 | 1236.2 | 1666.9 | 1870.4 |  2581.0 | 2637.5 | 2650.8 | 2650.8 |
| 4.0 |   19.0   |   15.0   | 1568.1 | 1648.3 | 2222.5 | 2493.8 |  3441.3 | 3516.7 | 3534.4 | 3534.4 |
| 5.0 |   23.0   |   19.0   | 1960.1 | 2060.4 | 2778.1 | 3117.3 |  4301.6 | 4395.9 | 4418.1 | 4418.1 |
| 6.0 |   27.0   |   23.0   | 2352.2 | 2472.5 | 3333.8 | 3740.7 |  5162.0 | 5275.0 | 5301.7 | 5301.7 |
| 7.0 |   31.0   |   27.0   | 2744.2 | 2884.5 | 3889.4 | 4364.2 |  6022.3 | 6154.2 | 6185.3 | 6185.3 |
| 8.0 |   35.0   |   31.0   | 3136.2 | 3296.6 | 4445.0 | 4987.6 |  6882.6 | 7033.4 | 7068.9 | 7068.9 |
| 9.0 |   39.0   |   35.0   | 3528.2 | 3708.7 | 5000.6 | 5611.1 |  7742.9 | 7912.6 | 7952.5 | 7952.5 |

##### Double Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |     3.5 |   5.125 |    5.25 |     5.5 |     7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|--------:|--------:|--------:|--------:|--------:|
| 2.0 |   11.0   |   7.0    | 1058.3 | 1234.7 | 2116.7 |  2469.5 |  3534.4 |  3534.4 |  3534.4 |  3534.4 |
| 3.0 |   15.0   |   11.0   | 1587.5 | 1852.1 | 3175.0 |  3704.2 |  5301.7 |  5301.7 |  5301.7 |  5301.7 |
| 4.0 |   19.0   |   15.0   | 2116.7 | 2469.5 | 4233.4 |  4939.0 |  7068.9 |  7068.9 |  7068.9 |  7068.9 |
| 5.0 |   23.0   |   19.0   | 2645.9 | 3086.8 | 5291.7 |  6173.7 |  8836.1 |  8836.1 |  8836.1 |  8836.1 |
| 6.0 |   27.0   |   23.0   | 3175.0 | 3704.2 | 6350.1 |  7408.4 | 10603.3 | 10603.3 | 10603.3 | 10603.3 |
| 7.0 |   31.0   |   27.0   | 3704.2 | 4321.6 | 7408.4 |  8643.2 | 12370.5 | 12370.5 | 12370.5 | 12370.5 |
| 8.0 |   35.0   |   31.0   | 4233.4 | 4939.0 | 8466.8 |  9877.9 | 14137.8 | 14137.8 | 14137.8 | 14137.8 |
| 9.0 |   39.0   |   35.0   | 4762.6 | 5556.3 | 9525.1 | 11112.6 | 15905.0 | 15905.0 | 15905.0 | 15905.0 |

### 1.000" Diameter

#### 2.50" oc. Spacing

##### Single Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   10.5   |   5.5    |  540.3 |  565.8 |  753.6 |  843.4 |  1158.7 | 1183.8 | 1234.4 | 1371.8 |
| 3.0 |   13.0   |   8.0    |  810.5 |  848.7 | 1130.4 | 1265.1 |  1738.0 | 1775.7 | 1851.5 | 2057.7 |
| 4.0 |   15.5   |   10.5   | 1080.7 | 1131.5 | 1507.2 | 1686.8 |  2317.3 | 2367.6 | 2468.7 | 2743.5 |
| 5.0 |   18.0   |   13.0   | 1350.9 | 1414.4 | 1884.0 | 2108.5 |  2896.7 | 2959.5 | 3085.9 | 3429.4 |
| 6.0 |   20.5   |   15.5   | 1621.0 | 1697.3 | 2260.8 | 2530.2 |  3476.0 | 3551.4 | 3703.1 | 4115.3 |
| 7.0 |   23.0   |   18.0   | 1891.2 | 1980.2 | 2637.6 | 2951.9 |  4055.4 | 4143.4 | 4320.3 | 4801.2 |
| 8.0 |   25.5   |   20.5   | 2161.4 | 2263.1 | 3014.4 | 3373.6 |  4634.7 | 4735.3 | 4937.4 | 5487.1 |
| 9.0 |   28.0   |   23.0   | 2431.5 | 2546.0 | 3391.1 | 3795.3 |  5214.0 | 5327.2 | 5554.6 | 6173.0 |

##### Double Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |    5.25 |     5.5 |     7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|--------:|--------:|--------:|
| 2.0 |   10.5   |   5.5    |  709.2 |  827.4 | 1418.3 | 1654.7 |  2423.0 |  2482.1 |  2600.3 |  2743.5 |
| 3.0 |   13.0   |   8.0    | 1063.8 | 1241.0 | 2127.5 | 2482.1 |  3634.5 |  3723.1 |  3900.4 |  4115.3 |
| 4.0 |   15.5   |   10.5   | 1418.3 | 1654.7 | 2836.7 | 3309.5 |  4846.0 |  4964.2 |  5200.6 |  5487.1 |
| 5.0 |   18.0   |   13.0   | 1772.9 | 2068.4 | 3545.9 | 4136.8 |  6057.5 |  6205.2 |  6500.7 |  6858.9 |
| 6.0 |   20.5   |   15.5   | 2127.5 | 2482.1 | 4255.0 | 4964.2 |  7269.0 |  7446.3 |  7800.9 |  8230.6 |
| 7.0 |   23.0   |   18.0   | 2482.1 | 2895.8 | 4964.2 | 5791.6 |  8480.5 |  8687.3 |  9101.0 |  9602.4 |
| 8.0 |   25.5   |   20.5   | 2836.7 | 3309.5 | 5673.4 | 6618.9 |  9692.0 |  9928.4 | 10401.2 | 10974.2 |
| 9.0 |   28.0   |   23.0   | 3191.3 | 3723.1 | 6382.5 | 7446.3 | 10903.5 | 11169.4 | 11701.3 | 12346.0 |

#### 3.00" oc. Spacing

##### Single Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   11.0   |   6.0    |  648.4 |  678.9 |  904.3 | 1012.1 |  1390.4 | 1420.6 | 1481.2 | 1646.1 |
| 3.0 |   14.0   |   9.0    |  972.6 | 1018.4 | 1356.5 | 1518.1 |  2085.6 | 2130.9 | 2221.8 | 2469.2 |
| 4.0 |   17.0   |   12.0   | 1296.8 | 1357.9 | 1808.6 | 2024.2 |  2780.8 | 2841.2 | 2962.5 | 3292.3 |
| 5.0 |   20.0   |   15.0   | 1621.0 | 1697.3 | 2260.8 | 2530.2 |  3476.0 | 3551.4 | 3703.1 | 4115.3 |
| 6.0 |   23.0   |   18.0   | 1945.2 | 2036.8 | 2712.9 | 3036.3 |  4171.2 | 4261.7 | 4443.7 | 4938.4 |
| 7.0 |   26.0   |   21.0   | 2269.4 | 2376.3 | 3165.1 | 3542.3 |  4866.4 | 4972.0 | 5184.3 | 5761.4 |
| 8.0 |   29.0   |   24.0   | 2593.7 | 2715.7 | 3617.2 | 4048.3 |  5561.6 | 5682.3 | 5924.9 | 6584.5 |
| 9.0 |   32.0   |   27.0   | 2917.9 | 3055.2 | 4069.4 | 4554.4 |  6256.8 | 6392.6 | 6665.5 | 7407.6 |

##### Double Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |    5.25 |     5.5 |     7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|--------:|--------:|--------:|
| 2.0 |   11.0   |   6.0    |  851.0 |  992.8 | 1702.0 | 1985.7 |  2907.6 |  2978.5 |  3120.4 |  3292.3 |
| 3.0 |   14.0   |   9.0    | 1276.5 | 1489.3 | 2553.0 | 2978.5 |  4361.4 |  4467.8 |  4680.5 |  4938.4 |
| 4.0 |   17.0   |   12.0   | 1702.0 | 1985.7 | 3404.0 | 3971.4 |  5815.2 |  5957.0 |  6240.7 |  6584.5 |
| 5.0 |   20.0   |   15.0   | 2127.5 | 2482.1 | 4255.0 | 4964.2 |  7269.0 |  7446.3 |  7800.9 |  8230.6 |
| 6.0 |   23.0   |   18.0   | 2553.0 | 2978.5 | 5106.0 | 5957.0 |  8722.8 |  8935.6 |  9361.1 |  9876.8 |
| 7.0 |   26.0   |   21.0   | 2978.5 | 3474.9 | 5957.0 | 6949.9 | 10176.6 | 10424.8 | 10921.2 | 11522.9 |
| 8.0 |   29.0   |   24.0   | 3404.0 | 3971.4 | 6808.0 | 7942.7 | 11630.4 | 11914.1 | 12481.4 | 13169.0 |
| 9.0 |   32.0   |   27.0   | 3829.5 | 4467.8 | 7659.0 | 8935.6 | 13084.2 | 13403.3 | 14041.6 | 14815.1 |

#### 4.00" oc. Spacing

##### Single Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |    3.0 |    3.5 |   5.125 |   5.25 |    5.5 |    7.0 |
|:---:|:--------:|:--------:|-------:|-------:|-------:|-------:|--------:|-------:|-------:|-------:|
| 2.0 |   12.0   |   7.0    |  864.6 |  905.2 | 1205.7 | 1349.4 |  1853.9 | 1894.1 | 1975.0 | 2194.8 |
| 3.0 |   16.0   |   11.0   | 1296.8 | 1357.9 | 1808.6 | 2024.2 |  2780.8 | 2841.2 | 2962.5 | 3292.3 |
| 4.0 |   20.0   |   15.0   | 1729.1 | 1810.5 | 2411.5 | 2698.9 |  3707.7 | 3788.2 | 3949.9 | 4389.7 |
| 5.0 |   24.0   |   19.0   | 2161.4 | 2263.1 | 3014.4 | 3373.6 |  4634.7 | 4735.3 | 4937.4 | 5487.1 |
| 6.0 |   28.0   |   23.0   | 2593.7 | 2715.7 | 3617.2 | 4048.3 |  5561.6 | 5682.3 | 5924.9 | 6584.5 |
| 7.0 |   32.0   |   27.0   | 3025.9 | 3168.3 | 4220.1 | 4723.1 |  6488.6 | 6629.4 | 6912.4 | 7681.9 |
| 8.0 |   36.0   |   31.0   | 3458.2 | 3621.0 | 4823.0 | 5397.8 |  7415.5 | 7576.4 | 7899.9 | 8779.3 |
| 9.0 |   40.0   |   35.0   | 3890.5 | 4073.6 | 5425.8 | 6072.5 |  8342.4 | 8523.5 | 8887.4 | 9876.8 |

##### Double Shear

|  n  |  h_beam  |  h_clip  |    1.5 |   1.75 |     3.0 |     3.5 |   5.125 |    5.25 |     5.5 |     7.0 |
|:---:|:--------:|:--------:|-------:|-------:|--------:|--------:|--------:|--------:|--------:|--------:|
| 2.0 |   12.0   |   7.0    | 1134.7 | 1323.8 |  2269.3 |  2647.6 |  3876.8 |  3971.4 |  4160.5 |  4389.7 |
| 3.0 |   16.0   |   11.0   | 1702.0 | 1985.7 |  3404.0 |  3971.4 |  5815.2 |  5957.0 |  6240.7 |  6584.5 |
| 4.0 |   20.0   |   15.0   | 2269.3 | 2647.6 |  4538.7 |  5295.1 |  7753.6 |  7942.7 |  8320.9 |  8779.3 |
| 5.0 |   24.0   |   19.0   | 2836.7 | 3309.5 |  5673.4 |  6618.9 |  9692.0 |  9928.4 | 10401.2 | 10974.2 |
| 6.0 |   28.0   |   23.0   | 3404.0 | 3971.4 |  6808.0 |  7942.7 | 11630.4 | 11914.1 | 12481.4 | 13169.0 |
| 7.0 |   32.0   |   27.0   | 3971.4 | 4633.3 |  7942.7 |  9266.5 | 13568.8 | 13899.8 | 14561.6 | 15363.8 |
| 8.0 |   36.0   |   31.0   | 4538.7 | 5295.1 |  9077.4 | 10590.3 | 15507.2 | 15885.4 | 16641.9 | 17558.7 |
| 9.0 |   40.0   |   35.0   | 5106.0 | 5957.0 | 10212.1 | 11914.1 | 17445.6 | 17871.1 | 18722.1 | 19753.5 |

